In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Masking
import matplotlib.pyplot as plt

# Load and preprocess the dataset

In [ ]:
# Load the CSV data into a pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/Stock price predict(Saify)/SANDP500.csv')

# Sort the DataFrame by the "Date" column in ascending order
df.sort_values(by='Date', inplace=True)

# Display the first few rows of the data to get an initial view
print(df.head())

          Date      Open      High       Low     Close Adj_Close  \
5434  1-Apr-02  1,147.39  1,147.84  1,132.87  1,146.54  1,146.54   
5181  1-Apr-03    848.18    861.28    847.85    858.48    858.48   
4928  1-Apr-04  1,126.21  1,135.67  1,126.20  1,132.17  1,132.17   
4677  1-Apr-05  1,180.59  1,189.80  1,169.91  1,172.92  1,172.92   
3923  1-Apr-08  1,326.41  1,370.18  1,326.41  1,370.18  1,370.18   

             Volume  
5434  1,050,900,000  
5181  1,461,600,000  
4928  1,560,700,000  
4677  2,168,690,000  
3923  4,745,120,000  


In [ ]:
df[df['Volume'] == "-"]

,Date,Open,High,Low,Close,Adj_Close,Volume
11,13-Oct-23,"4,360.49","4,377.10","4,311.97","4,327.78","4,327.78",-


In [ ]:
df = df[df["Volume"].str.contains("-") == False]

In [ ]:
df[df['Volume'] == "-"]

,Date,Open,High,Low,Close,Adj_Close,Volume


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5722 entries, 5434 to 286
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date       5722 non-null   object
 1   Open       5722 non-null   object
 2   High       5722 non-null   object
 3   Low        5722 non-null   object
 4   Close      5722 non-null   object
 5   Adj_Close  5722 non-null   object
 6   Volume     5722 non-null   object
dtypes: object(7)
memory usage: 357.6+ KB


In [ ]:
first_close_value_str = str(df['Close'].iloc[0])
# Remove commas from both 'Close' and 'Volume' columns and convert them to numeric
if ',' in first_close_value_str:
  # Filter out rows with missing values in the "Volume" column
  # df = df.dropna(subset=['Volume'])

  columns_to_convert = ['Close', 'Open', 'High', 'Low', 'Adj_Close']

  for column in columns_to_convert:
      df[column] = df[column].str.replace(',', '').astype(float)

  # Convert the "Date" column to a datetime column
  df['Date'] = pd.to_datetime(df['Date'])

  # Set "Date" as the index
  df.set_index('Date', inplace=True)
  print(df.sample(5))

else:
  print(df.head(5))

               Open     High      Low    Close  Adj_Close         Volume
Date                                                                    
2023-01-05  3839.74  3839.74  3802.42  3808.10    3808.10  3,893,450,000
2001-02-08  1341.10  1350.32  1332.42  1332.53    1332.53  1,107,200,000
2014-01-28  1783.00  1793.87  1779.49  1792.50    1792.50  3,437,830,000
2018-11-19  2730.74  2733.16  2681.09  2690.73    2690.73  3,806,940,000
2018-12-12  2658.23  2685.44  2650.26  2651.07    2651.07  4,029,300,000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5722 entries, 2002-04-01 to 2022-09-09
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       5722 non-null   float64
 1   High       5722 non-null   float64
 2   Low        5722 non-null   float64
 3   Close      5722 non-null   float64
 4   Adj_Close  5722 non-null   float64
 5   Volume     5722 non-null   object 
dtypes: float64(5), object(1)
memory usage: 312.9+ KB


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5722 entries, 2002-04-01 to 2022-09-09
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       5722 non-null   float64
 1   High       5722 non-null   float64
 2   Low        5722 non-null   float64
 3   Close      5722 non-null   float64
 4   Adj_Close  5722 non-null   float64
 5   Volume     5722 non-null   object 
dtypes: float64(5), object(1)
memory usage: 312.9+ KB


In [ ]:
df.head()

,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2002-04-01,1147.39,1147.84,1132.87,1146.54,1146.54,"1,050,900,000"
2003-04-01,848.18,861.28,847.85,858.48,858.48,"1,461,600,000"
2004-04-01,1126.21,1135.67,1126.20,1132.17,1132.17,"1,560,700,000"
2005-04-01,1180.59,1189.80,1169.91,1172.92,1172.92,"2,168,690,000"
2008-04-01,1326.41,1370.18,1326.41,1370.18,1370.18,"4,745,120,000"


In [ ]:
df["Volume"] = df["Volume"].str.replace('.','', regex=True).str.replace(',', '', regex=True).astype(float)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5722 entries, 2002-04-01 to 2022-09-09
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       5722 non-null   float64
 1   High       5722 non-null   float64
 2   Low        5722 non-null   float64
 3   Close      5722 non-null   float64
 4   Adj_Close  5722 non-null   float64
 5   Volume     5722 non-null   float64
dtypes: float64(6)
memory usage: 312.9 KB


In [ ]:
# saving cleaned dataframe
df.to_csv('/content/drive/MyDrive/Stock price predict(Saify)/cleand_SANDP500.csv')


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5722 entries, 2002-04-01 to 2022-09-09
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       5722 non-null   float64
 1   High       5722 non-null   float64
 2   Low        5722 non-null   float64
 3   Close      5722 non-null   float64
 4   Adj_Close  5722 non-null   float64
 5   Volume     5722 non-null   float64
dtypes: float64(6)
memory usage: 312.9 KB
